In [1]:
from utils.utils import get_circs_from_exp_design, get_exp_design, save_dataset, clear_and_mk_files
from utils.parse import Parse
from qiskit import IBMQ
import numpy as np
from qiskit import *
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.tools.monitor import job_monitor
from tqdm import tqdm

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-skku', group='snu', project='snu-students')
backend = provider.get_backend('ibmq_manhattan')

In [2]:
GST_type = "smq2Q_XYICPHASE"
n_qubit = 2
num_shots_per_point = 1024
max_max_length = 16
meas_key = ['00', '01', '10', '11']

In [3]:
clear_and_mk_files(GST_type, max_max_length)

smq2Q_XYICPHASE folder is removed
smq2Q_XYICPHASE folder with empty dataset.txt is reconstructed


In [4]:
exp_design = get_exp_design(max_max_length, GST_type)
circs = get_circs_from_exp_design(exp_design, n_qubit)

In [5]:
circs = transpile(circs, backend=backend)
job_manager = IBMQJobManager()
job_set_foo = job_manager.run(circs, backend=backend, name='foo', meas_level = 2, shots = num_shots_per_point)

In [6]:
for job in tqdm(job_set_foo.jobs()):
    job_monitor(job)

  0%|          | 0/11 [00:00<?, ?it/s]

Job Status: job is actively running

  9%|▉         | 1/11 [05:44<57:20, 344.01s/it]

Job Status: job has successfully run
Job Status: job is actively running

 18%|█▊        | 2/11 [11:56<52:52, 352.52s/it]

...

Job Status: job has successfully run
Job Status: job is actively running

 91%|█████████ | 10/11 [1:20:19<07:17, 437.10s/it]

Job Status: job has successfully run


100%|██████████| 11/11 [1:20:20<00:00, 438.24s/it]

Job Status: job has successfully run


In [22]:
results = job_set_foo.results().combine_results()
meas_dict = [results.get_counts(i) for i in range(len(results.results))]

In [29]:
# Warning: 01 qubit means 1, 0 are measured in qubit 0, 1 in qiskit.
meas_dict_pygsti = []
for D in meas_dict:
    D_pygsti = {}
    for k in D:
        if k=='00 00':
            D_pygsti['00'] = D[k]
        elif k=='10 00':
            D_pygsti['01'] = D[k]
        elif k=='01 00':
            D_pygsti['10'] = D[k]
        elif k=='11 00':
            D_pygsti['11'] = D[k]
    meas_dict_pygsti.append(D_pygsti)
for D in meas_dict_pygsti:
    for key in meas_key:
        if key not in D:
            D[key] = 0

In [31]:
save_dataset(exp_design, meas_dict_pygsti, meas_key, GST_type)

In [ ]:
idx = 0
print(meas_key)
print(len(circs), len(meas_dict),len(exp_design.all_circuits_needing_data))
print(circs[idx])
print(meas_dict_pygsti[idx])
print(exp_design.all_circuits_needing_data[idx].str)